In [2]:
:dep ndarray = "0.15.0"
:dep rand = "0.6.5"
:dep plotly = "0.6.0"
:dep itertools-num

In [3]:
extern crate rand;
use rand::distributions::{Normal, Distribution};

In [4]:
extern crate plotly;
use plotly::{Plot, Layout, Scatter};

# Generate data

In [5]:
struct RandomMotionParameters {
    initial_state : ndarray::Array1<f64>,
    motion_noise : f64,
    measurement_noise : f64
}

fn generate_data(point_count: i64, parameters: &RandomMotionParameters, time_delta: f64)
    -> (Vec<ndarray::Array1<f64>>, Vec<ndarray::Array1<f64>>) {
    let mut state: ndarray::Array1<f64> = parameters.initial_state.clone();
    let motion_distribution = Normal::new(0., parameters.motion_noise);
    let measurement_distribution = Normal::new(0., parameters.motion_noise);
    let mut measurements: Vec<ndarray::Array1<f64>> = Vec::new();
    let mut states: Vec<ndarray::Array1<f64>> = Vec::new();
    for _ in 0..point_count {
        let acceleration = motion_distribution.sample(&mut rand::thread_rng());
        state = ndarray::arr1(&[
            state[0] + state[1] * time_delta + acceleration * time_delta.powi(2) / 2.,
            state[1] + acceleration * time_delta
        ]);
        let measurement:ndarray::Array1<f64> = ndarray::arr1(&[
            state[0] + measurement_distribution.sample(&mut rand::thread_rng())
        ]);
        measurements.push(measurement.clone());
        states.push(state.clone());
    }
    (measurements, states)
}


In [11]:
let time_delta = 0.1;
let points_count:i64 = 500;
let times: Vec<f64> =  (0..points_count).map(|x: i64| -> f64 {x as f64 * time_delta}).collect();
let initial_state : ndarray::Array1<f64> = ndarray::arr1(&[0., 0.]);
let parameters = RandomMotionParameters{
    initial_state: initial_state,
    measurement_noise: 1.,
    motion_noise: 1.
};
let result:(Vec<ndarray::Array1<f64>>, Vec<ndarray::Array1<f64>>) = generate_data(points_count, &parameters, time_delta);
let measurements: Vec<ndarray::Array1<f64>> = result.0;
let states: Vec<ndarray::Array1<f64>> = result.1;
let measurement_data: Vec<f64> = measurements.iter().map(|x| -> f64 {x[0]}).collect();
let ground_truth_data: Vec<f64> = states.iter().map(|x| -> f64 {x[0]}).collect();

## Plot

In [12]:
let trace1 = Scatter::new(times.clone(), ground_truth_data.clone());
let trace2 = Scatter::new(times.clone(), measurement_data.clone());
let mut plot = Plot::new();
plot.add_trace(trace1);
plot.add_trace(trace2);
plot.notebook_display();

In [8]:
use std::env::current_dir;
let path = std::env::current_dir()?;

In [9]:
let static_files = format!("{}\\src\\user.rs", path.display());

In [10]:
let user_path = static_files.as_bytes();

Error: `static_files` does not live long enough

Error: cannot move out of `static_files` because it is borrowed